**This notebook is the executable version of lab note 3.
It answers the following questions:**

Finally, we answer 4 items:

    1. Are successful creators more connected to high-outdegree users than do unsuccessful creators?
    2. Are mavens more connected to successful creators than to unsuccessful creators?
    3. Do successful creators send more non-follow actions towards mavens than to zombies?
    4. Do successful creators send more non-follow actions towards mavens than to stars?

    

In [1]:
#Run parameters
#used to control every run. Can be user to perfom sensitivity checks
path_dir = r"/Users/../Volumes/Raw/"

low_success = 0.5 #below the median: unsuccessful
high_success = 0.9 #top 10% creators with more followers are deemed successful

low_user_outdegree = 0.25 
high_user_outdegree = 0.75
low_user_activity = 0.25 
high_user_activity = 0.75 

activity_filter = 0
days_delta = 7

In [2]:
import sys  
import pickle
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
import src.utils
from collections import Counter
from src.utils import import_dta, import_tracks_dta,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob, stripplot_prob,\
reaction_probability, follower_list, filter_quantile, sample_creators_music,\
gen_outbound_creators
import numpy as np
import datetime
import pandas as pd
from tqdm import tqdm
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import os
from statsmodels.stats.proportion import proportions_ztest
from datetime import datetime, timedelta


In [3]:
def process_date(date):
    '''convert date format like '2013-w09' to '2013-03-04', i.e. the first day of that week'''
    year = date[0:4]
    week = date[6:]
    day = "1"
    date = "{}-{}-1".format(year, week)
    dt = datetime.strptime(date, "%Y-%W-%w")
    return dt

# Data Imports

We start by importing the raw data.  `follows_sent`, `comments_sent`, `shares_sent`, `likes_sent` and `messages_sent` contains data pn the promotional activities that the 35k users tracked in the dataset directed to other users. It includes the `user_id`, the `fan_id` and the `date_sent` which identifies the date when the prom. activity was sent. `users_info_1st` shows the type of user (creator or non-creator, which is identified by a blank) and the date the user entered the platform, for every user that sent or received prom. activities from any of the 35k users tracked in this dataset, while `users_info` contains the same information, but pertaining to the 35k users themselves.

`follows_received` contains information on the follows received by the 35k users and will be used to generate the successful/unsuccessful groups of content creators.

In [161]:
#affiliations :follows
#favoritings :likes

#used in filtering:
path_dir = r"/Users/../Volumes/Raw/"
tracks = import_tracks_dta(path_dir, "12sample_tracks.dta");

#these are the actions sent to 
follows_sent = import_dta(path_dir, "12sample_affiliations_sent.dta");
comments_sent = import_dta(path_dir, "12sample_comments_made.dta");
shares_sent = import_dta(path_dir, "12sample_reposts_made.dta");
likes_sent = import_dta(path_dir, "12sample_favoritings_made.dta");
messages_sent = import_dta(path_dir, "12sample_messages_sent.dta");

#Used to track information on the 1st degree connections
user_info_1st = import_dta(path_dir, "12sample_1st_deg_user_infos.dta");
user_info_1st.columns = ['user_id', 'type', 'entered_platform'];
user_info = import_dta(path_dir, "12sample_user_infos.dta");

#Used to compute creator's success measure
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta");

%%%%%%%%%% 12sample_tracks.dta %%%%%%%%%%
(56262, 7)
%%%%%%%%%% 12sample_affiliations_sent.dta %%%%%%%%%%
(800913, 3)
%%%%%%%%%% 12sample_comments_made.dta %%%%%%%%%%
(29258, 4)
%%%%%%%%%% 12sample_reposts_made.dta %%%%%%%%%%
(179329, 4)
%%%%%%%%%% 12sample_favoritings_made.dta %%%%%%%%%%
(527701, 4)
%%%%%%%%%% 12sample_messages_sent.dta %%%%%%%%%%
(11091, 3)
%%%%%%%%%% 12sample_1st_deg_user_infos.dta %%%%%%%%%%
(670746, 3)
%%%%%%%%%% 12sample_user_infos.dta %%%%%%%%%%
(35000, 3)
%%%%%%%%%% 12sample_affiliations_received.dta %%%%%%%%%%
(432503, 3)


Indegree and outdegree information.

The function below import the outdegree dataset. Because the raw version of those dataset are too large to be processed in memory, we preprocessed them in a separate script.

In [5]:
# Aggregates preprocessed outdegree of 1st degree users
def import_outdegree(path='/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/'):
    d = {}
    for i in range(6):
       d[str(i)] = pd.read_pickle(os.path.join(path,'{}.pkl'.format(i))) 
       d[str(i)]['created_at'] =  pd.to_datetime(d[str(i)]['created_at'])
       d[str(i)]['created_at'] = pd.to_datetime(d[str(i)]['created_at']).dt.floor('d')
       d[str(i)] = d[str(i)].groupby(['sender_id', 'created_at'], as_index = False).size() 
    
    data_outdegree = pd.concat([d['0'], d['1'], d['2'], d['3'], d['4'], d['5']])
    #data_outdegree.set_index('created_at', inplace = True)
    return data_outdegree

In [6]:
data_outdegree = import_outdegree()

In [7]:
#data_outdegree = data_outdegree.groupby(['sender_id','created_at'], as_index = False).size()

# Preprocessing

## Creator ids, successful and unsucessful creators

Next, we define three lists of ids: one with the ids from the content creators, according to the `users_info` table, one with the ids of successful creators and the last one with the ids of the unsuccessful ones.

Let's start with a list of the id of creators. We also create a dataset with containing information on creators only.

In [8]:
mask = (tracks.track_available == 1) & (tracks.public == 't')
creator_ids = tracks[mask].user_id.unique()

creators = tracks[(tracks.track_available == 1) & (tracks.public == 't')]

#mask = user_info.type == 'creator'
#creator_ids = user_info[mask].user_id.unique()

#creators = user_info[user_info.type == 'creator']

## Putting together a dataset with the promotional activities made by content creators.

The function `gen_actions_sent_df` creates a dataframe with all the promotional activities that content creators sent to users.

In [9]:
def gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent, messages_sent, creator_ids = creator_ids):
    '''
    Creates dataframe containing the actions that content creators send to users.
        Attributes:
                    follows_sent:  dataframe with the follows sent by the 35k users.
                    shares_sent:   dataframe with the shares sent by the 35k users.
                    likes_sent:    dataframe with the likes sent by the 35k users.
                    comments_sent: dataframe with the comments sent by the 35k users.
                    messages_sent: dataframe with the messages sent by the 35k users.
                    creator_ids:   list with content creator ids. If not none, is used to
                                   filter out activities from non creators.
    '''
    
    follows_sent['outbound_activity'] = 'follow'
    follows_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'song_id' in shares_sent.columns:
        shares_sent.drop(columns=["song_id"])
    shares_sent = shares_sent[['reposter_id', "owner_id", 'created_at']]
    shares_sent['outbound_activity'] = 'share'
    shares_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in likes_sent.columns:
        likes_sent.drop(columns=["track_id"], inplace=True)
    likes_sent['outbound_activity'] = 'like'
    likes_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in comments_sent.columns:
        comments_sent.drop(columns=["track_id"], inplace=True)
    comments_sent['outbound_activity'] = 'comment'
    comments_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    messages_sent["outbound_activity"] = 'message'
    messages_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']
    df = pd.concat([follows_sent, shares_sent, likes_sent, comments_sent, messages_sent])


    if type(creator_ids) == numpy.ndarray:
        df = df[df['user_id'].isin(creator_ids)]
        
    df['week_yr'] = df.date_sent.dt.strftime('%Y-w%U')
    df = df.loc[df['user_id'] != df['fan_id'],:]

    return df

In [10]:
actions_sent = gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent,
                                     messages_sent, creator_ids = None)
actions_sent = actions_sent.loc[actions_sent.user_id.isin(creators.user_id.unique())]

In [11]:
active_users_ids = actions_sent.groupby('user_id', as_index = False).size()
mask = active_users_ids['size']>= activity_filter
active_users_ids = active_users_ids[mask].user_id.unique()

In [165]:
def successful_creators_followers(follows_received, base_date = datetime(2016, 5, 30, 0, 0), perc1 = None, perc2 = None, subset_creators = None):
    '''Classifies content creators in successful or unsuccessfull
        Arguments:
                    follows_received: dataframe containing the follows received by content creators
                    base date:        date, in datetime.datetime(YYYY, M, DD, H, M) format, in which the number 
                                      of followers per creator is calculated.
                    perc1:            the threshold used to classify unsuccessful content creators. Creator having 
                                      total followers below the number dictated by this threshold, at the base date,
                                      are classified as unsuccessful 
                    perc2:            the threshold used to classify successful content creators. Creator having 
                                      total followers above the number dictated by this threshold, at the base date,
                                      are classified as successful
                    subset_creators:  a pd.DataFrame containing the creators. If is it available, it will be used to 
                                      filter out non creators and to make sure creators with 0 followers are part of
                                      the resulting dataset.
        
    '''
    print(base_date)

    if 'inbound_activity' not in follows_received.columns:
        follows_received.columns = ['fan_id', 'user_id', 'date_sent']

    mask = (follows_received['date_sent'] < base_date)

    df = follows_received[mask].groupby('user_id', as_index=False).agg({'fan_id': pd.Series.nunique})
    df.columns = ['user_id', 'followers']

    
    if type(subset_creators) == pd.DataFrame:
        print('subsetting...')
        df.set_index('user_id', inplace = True)
        df = df.reindex(subset_creators.user_id.unique())
        df.fillna(0, inplace = True)
        df.reset_index(inplace = True)
        df.columns = ['user_id', 'followers']
        
    mask = df.user_id.isin(active_users_ids)
    df = df[mask]

    low = np.quantile(df.followers, perc1)
    high = np.quantile(df.followers, perc2)

    print("High influencer boundary: {}".format(high))
    print("Low influencer boundary: {}".format(low))

    mask = (df["followers"] <= low) | (df["followers"] >= high)
    
    unsuccessful_creator_ids = df.loc[df["followers"] <= low].user_id.unique()
    successful_creator_ids = df.loc[df["followers"] >= high].user_id.unique()

    return unsuccessful_creator_ids, successful_creator_ids


In [166]:
unsuccessful_ids, successful_ids = successful_creators_followers(follows_received, 
                                                        perc1 = low_success, perc2 = high_success, subset_creators = creators)


2016-05-30 00:00:00
subsetting...
High influencer boundary: 81.0
Low influencer boundary: 13.0


In [14]:
print(len(unsuccessful_ids))
print(len(successful_ids))

1959
389


In [15]:
creators.user_id.nunique()

4604

## Filter only actions that were sent to non-fans

We merge the `actions_sent` dataset with a table containing the date each fan started following the creator.

In [168]:
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follows_received[["fan_id", "user_id", "date_sent"]]
followers.columns = ["fan_id", "user_id", "follower_since"]

actions_sent = actions_sent.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')

Since we are interested in acquisition campaings, we need to produce a dataset that exclude actions targetting fans.
We do that using filters based on the date of the action and the date that the user became a fan of the content creator. The resulting dataframe is named `actions_sent_non_fans`.

We then filter only actions that happened before the user follows the content creator.

In [17]:
mask = (actions_sent.date_sent < actions_sent.follower_since) | (actions_sent.follower_since.isnull())
actions_sent_non_fans =  actions_sent[mask]
actions_sent_non_fans['week_yr_date'] = actions_sent_non_fans.week_yr.apply(lambda x: process_date(x))

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_67214/3855957735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_non_fans['week_yr_date'] = actions_sent_non_fans.week_yr.apply(lambda x: process_date(x))


## Outdegree level

Originally, we only have outdegree information on users that follow at least one user. The function below inputs an outdegree of 0 to users that are following anyone.

In [18]:
#CHANGE DOCUMENTATION
def outdegree_level(data, date, user_info = user_info_1st):
    
    '''
    This function returns the membership table at date equals `date`. Every user that interacted with the 35k tracked 
    users and entered the platform before `date` is present in the table, even if it has indegree 0.
    arguments:
              data:           the indegree dataset.
              user_info:      the dataset containing all the users that interacted with the 35k users tracked.
    '''
    
    data = data[data.created_at.dt.floor('d')<=date]
    data = data.groupby('sender_id').agg({'size':'sum'}).compute()
    
    #merge with user info to obtain users that are not followed by anyone at the current date
    data = user_info_1st.merge(data, left_on = 'user_id', right_on = 'sender_id', how= 'outer')
    data.loc[data['size'].isnull(), 'size'] = 0
    data = data[['user_id', 'size', 'entered_platform']].set_index('user_id')
    
    #filter out users that didnt exist in the current date
    mask = data['entered_platform'].dt.floor('d') <= date
    data = data.loc[mask]
    
    mask = (data['size']>0)
    data.loc[~mask, 'size'] = 0 
    
    return data

In [19]:
#dask is a python api with objects optimized for big data (user directed acyclic graphs). 
dask_outdegree = dd.from_pandas(data_outdegree, npartitions = 3)

In [20]:
last_day =  max(actions_sent.date_sent.dt.floor('d').unique())
user_outdegree = outdegree_level(dask_outdegree, last_day, user_info = user_info_1st)

Now we classify the creator in successful and unsuccessful, according to the threshold defined in the beggining of this notebook.

In [21]:
def user_outdegree_level(oudegree_data, perc1 = None, perc2 = None):
    '''Classifies content creators in successful or unsuccessfull
        Arguments:
                    oudegree_data:    dataframe containing the fans followers at 
                    perc1:            the threshold used to classify unsuccessful content creators. Creator having 
                                      total followers below the number dictated by this threshold, at the base date,
                                      are classified as unsuccessful 
                    perc2:            the threshold used to classify successful content creators. Creator having 
                                      total followers above the number dictated by this threshold, at the base date,
                                      are classified as successful
    '''

    df = oudegree_data.reset_index().iloc[:,:2]
    df.columns = ['user_id', 'followers']

    low = np.quantile(df.followers, perc1)
    high = np.quantile(df.followers, perc2)

    print("High outdegree boundary: {}".format(high))
    print("Low outdegree boundary: {}".format(low))
    
    low_outdegree_ids = df.loc[df["followers"] <= low].user_id.unique()
    high_outdegree_ids = df.loc[df["followers"] >= high].user_id.unique()
    
    
    df['outdegree_level'] = df.user_id.apply(
        lambda x: 'high' if x in high_outdegree_ids else ('low' if x in low_outdegree_ids else None))

    
    return df, low_outdegree_ids, high_outdegree_ids

In [22]:
user_outdegree_df, low_outdegree_ids, high_outdegree_ids = user_outdegree_level(user_outdegree,
                perc1 = low_user_outdegree, perc2 = high_user_outdegree)

High outdegree boundary: 162.0
Low outdegree boundary: 17.0


In the cell below, we create a list with unique ids from users that appear in the oudegree level table. This will later be uses to construct a flow-chart indicating how we lose data based on filters and operations.

In [23]:
receiver_ids = user_outdegree.index.unique()

## Non-follow Actions level

The activity level is defined as the number of actions performed by users. It is important to notice that we only observe actions targeting the 35k users that joined in march 2012. We consider this measure a proxy for the real activity level.

Let's begin by creating a dataset with all action received by those 35k users.

In [24]:
comments_received_35k = import_dta(path_dir, "12sample_comments_received.dta");
shares_received_35k = import_dta(path_dir, "12sample_reposts_received.dta");
likes_received_35k = import_dta(path_dir, "12sample_favoritings_received.dta");
messages_received_35k = import_dta(path_dir, "12sample_messages_received.dta");

if 'song_id' in shares_received_35k:
        shares_received_35k.drop(columns=["song_id"])
shares_received_35k = shares_received_35k[['reposter_id', "owner_id", 'created_at']]
shares_received_35k['outbound_activity'] = 'share'
shares_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in likes_received_35k:
        likes_received_35k = likes_received_35k.drop(columns=["track_id"])
likes_received_35k['outbound_activity'] = 'like'
likes_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in comments_received_35k:
        comments_received_35k = comments_received_35k.drop(columns=["track_id"])
comments_received_35k['outbound_activity'] = 'comment'
comments_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

messages_received_35k["outbound_activity"] = 'message'
messages_received_35k.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

user_activity_data_35k = pd.concat([shares_received_35k, likes_received_35k, comments_received_35k, messages_received_35k])

%%%%%%%%%% 12sample_comments_received.dta %%%%%%%%%%
(21386, 4)
%%%%%%%%%% 12sample_reposts_received.dta %%%%%%%%%%
(83013, 4)
%%%%%%%%%% 12sample_favoritings_received.dta %%%%%%%%%%
(286903, 4)
%%%%%%%%%% 12sample_messages_received.dta %%%%%%%%%%
(17364, 3)


In [25]:
path_dir_2 = r'/Users/../Volumes/Alter_outbound_activities/'

comments_received_c = import_dta(path_dir_2, "12sample_1st_deg_comments_made.dta");
shares_received_c = import_dta(path_dir_2, "12sample_1st_deg_reposts_made.dta");
likes_received_c = import_dta(path_dir_2, "12sample_1st_deg_favoritings_made.dta");
messages_received_c = import_dta(path_dir_2, "12sample_1st_deg_messages_sent.dta");

%%%%%%%%%% 12sample_1st_deg_comments_made.dta %%%%%%%%%%
(21463011, 4)
%%%%%%%%%% 12sample_1st_deg_reposts_made.dta %%%%%%%%%%
(18953640, 4)
%%%%%%%%%% 12sample_1st_deg_favoritings_made.dta %%%%%%%%%%
(86793370, 4)
%%%%%%%%%% 12sample_1st_deg_messages_sent.dta %%%%%%%%%%
(16824074, 3)


In [26]:
if 'song_id' in shares_received_c:
        shares_received_c.drop(columns=["song_id"])
shares_received_c = shares_received_c[['reposter_id', "owner_id", 'created_at']]
shares_received_c['inbound_activity'] = 'share'
shares_received_c.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in likes_received_c:
        likes_received_c = likes_received_c.drop(columns=["track_id"])
likes_received_c['inbound_activity'] = 'like'
likes_received_c.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in comments_received_c:
        comments_received_c = comments_received_c.drop(columns=["track_id"])
comments_received_c['inbound_activity'] = 'comment'
comments_received_c.columns_c = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

messages_received_c["outbound_activity"] = 'message'
messages_received_c.columns = ['user_id', 'fan_id', 'date_sent', 'inbound_activity']

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_67214/1613121866.py:15: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  comments_received_c.columns_c = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']


In [27]:
user_activity_data_c = pd.concat([shares_received_c, likes_received_c, comments_received_c, messages_received_c])

In [28]:
def user_activity_level(user_activity_data, receiver_ids, perc1 = None, perc2 = None):
    '''Classifies users based on the amount of non-follow activities
        Arguments:
                    user_activity_data:   dataframe containing the user activities 
                    base date:             date, in datetime.datetime(YYYY, M, DD, H, M) format, in which the number 
                                           of followers per creator is calculated.
                    perc1:                 the threshold used to classify unsuccessful content creators. Creator having 
                                           total followers below the number dictated by this threshold, at the base date,
                                           are classified as unsuccessful 
                    perc2:                 the threshold used to classify successful content creators. Creator having 
                                           total followers above the number dictated by this threshold, at the base date,
                                           are classified as successful
    '''

    df = user_activity_data.groupby('fan_id', as_index = True).size()
    df = df.reindex(receiver_ids)
    print(df.shape)
    df = df.reset_index()
    df.columns = ['user_id', 'activities_performed']
    
    df.loc[df.activities_performed.isna(), 'activities_performed'] = 0
    #classification should happen after this

    low = np.quantile(df.activities_performed, perc1)
    high = np.quantile(df.activities_performed, perc2)

    print("High activity boundary: {}".format(high))
    print("Low activity boundary: {}".format(low))
    
    low_activity_ids = df.loc[df["activities_performed"] <= low].user_id.unique()
    high_activity_ids = df.loc[df["activities_performed"] > high].user_id.unique()
    
    df['activity_level'] = df.user_id.apply(
    lambda x: 'high' if x in high_activity_ids else ('low' if x in low_activity_ids else None))

    return df, low_activity_ids, high_activity_ids

In [29]:
activity_level, low_activity_ids, high_activity_ids = user_activity_level(user_activity_data_c, receiver_ids, 
                perc1 = low_user_activity, perc2 = high_user_activity)

(670746,)
High activity boundary: 70.0
Low activity boundary: 1.0


Once more we create an object containing the unique ids of users in the resulting dataset. This will be used in a flow-chart, as explained.

In [30]:
activity_ids = np.append(low_activity_ids, high_activity_ids)

# Priori response probabilities

In [31]:
table_data = user_outdegree_df.merge(activity_level, left_on = 'user_id', right_on = 'user_id', how = 'inner')
table_data.groupby(['outdegree_level', 'activity_level']).size()
table_data['Type'] = table_data.apply(lambda x: 
    'f_a' if (x.outdegree_level == 'low') & (x.activity_level == 'high')
     else ('Hermit' if (x.outdegree_level == 'low') & (x.activity_level == 'low')
     else ('Observer' if (x.outdegree_level == 'high') & (x.activity_level == 'low')
     else ('w_a' if (x.outdegree_level == 'high') & (x.activity_level == 'high')
     else 'other'))), axis=1)

hermit_ids = table_data.loc[table_data.Type =='Hermit'].user_id.unique()
w_a_ids = table_data.loc[table_data.Type =='w_a'].user_id.unique()
f_a_ids = table_data.loc[table_data.Type =='f_a'].user_id.unique()
observer_ids = table_data.loc[table_data.Type =='Observer'].user_id.unique()

In [35]:
table_data.to_csv('user_types_ids.csv')

In [170]:
follow_follower_lists = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists_3.pkl')

def compute_indirect_return_v3(follower_lists, follows_received_df, indirect_return_df): 
    
    follows_received_df['date_sent'] = follows_received_df['date_sent'].dt.normalize()
    follows_received_df = follows_received_df.loc[(follows_received_df.user_id.isin(creator_ids))]
    follows_received_df = follows_received_df.loc[follows_received_df.fan_id.isin(follower_lists.index.unique())]
    
    for user_id in tqdm(follows_received_df.user_id.unique()):
        df_j = follows_received_df.loc[follows_received_df.user_id == user_id]
        
        for fan_id in follows_received_df.loc[follows_received_df.user_id == user_id].fan_id.unique():
            
            fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]
            
            start_date = df_j.loc[(df_j.user_id == user_id)&
                                              (df_j.fan_id == fan_id)].date_sent.iloc[0]
            end_date = datetime(2015, 7, 30)
    
            creator_followers_from_t = df_j.loc[df_j.date_sent >= start_date]\
                                       .groupby('user_id', as_index = False)['fan_id'].apply(np.array)[0]
                
            indirect_return_df.loc[(indirect_return_df['user_id'] == user_id)
                                   & (indirect_return_df['fan_id'] == fan_id), 'Follower_prop'] =\
            sum(np.in1d(creator_followers_from_t, fan_followers))/len(fan_followers)

In [171]:
indirect_return_df = follows_received.loc[(follows_received.user_id.isin(creator_ids))&
                                          (follows_received.fan_id.isin(follow_follower_lists.index.unique()))
                                          ,['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])

teste = compute_indirect_return_v3(follow_follower_lists, follows_received, indirect_return_df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3805/3805 [07:23<00:00,  8.57it/s]


In [172]:
indirect_return_df

indirect_return_df['user_type'] = indirect_return_df.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
indirect_return_df['creator_type'] = indirect_return_df.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

In [173]:
indirect_return_df.to_csv('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/all_creators_follower_prop.csv')

In [200]:
indirect_return_df.groupby(['user_type', 'creator_type']).agg({'Follower_prop': ['mean', 'std']})

Follower_prop          
                                mean       std
user_type creator_type                        
Hermit    other             0.087779  0.191391
          successful        0.094453  0.217115
          unsuccessful      0.064817  0.179834
Observer  other             0.025996  0.064873
          successful        0.013832  0.042443
          unsuccessful      0.007981  0.036871
f_a       other             0.052020  0.119501
          successful        0.049271  0.116733
          unsuccessful      0.011931  0.037543
other     other             0.039371  0.092539
          successful        0.033256  0.087659
          unsuccessful      0.021909  0.074919
w_a       other             0.005928  0.017859
          successful        0.013675  0.030687
          unsuccessful      0.002034  0.007292

In [106]:
indirect_return_df.groupby('user_type').size()

user_type
Hermit       2219
Observer     2384
f_a           830
other       66494
w_a         34682
dtype: int64

In [36]:
follows_received.

,fan_id,user_id,date_sent
0,2763368,37873923,2013-03-01 16:46:05
1,32678887,37876767,2013-03-01 16:48:22
2,37776943,37892700,2013-03-01 18:00:13
3,2880445,37866904,2013-03-01 19:45:43
4,3007254,37873923,2013-03-01 20:30:44
...,...,...,...
432498,14016269,37973896,2015-06-30 14:18:53
432499,10492751,38347478,2015-06-30 14:48:44
432500,90937808,38222799,2015-06-30 15:33:54
432501,4203669,37881808,2015-06-30 16:18:00


In [55]:
shares = user_activity_data_35k.loc[user_activity_data_35k.outbound_activity == 'share']

In [187]:
follows_received.columns = ['fan_id', 'user_id', 'follower_since']

In [190]:
shares_df = shares.merge(follows_received, on = ['user_id', 'fan_id'], how = 'left')
shares_df = shares_df.loc[shares_df.follower_since.isna() == False]

shares_df.sort_values('date_sent', inplace = True)
shares_df.drop_duplicates(subset=['user_id', 'fan_id'], keep='first', inplace = True)

In [191]:
shares_df

,fan_id,user_id,date_sent,outbound_activity,follower_since
9620,4074557,37851487,2013-03-01 09:28:19,share,2013-03-01
17020,2564143,37872188,2013-03-01 15:08:24,share,2013-03-01
2590,37883606,37873724,2013-03-01 16:10:28,share,2013-03-23
36106,2042445,37886585,2013-03-01 21:18:46,share,2013-03-01
43898,14503986,37886585,2013-03-01 21:22:09,share,2013-03-01
...,...,...,...,...,...
50993,9215506,38445740,2015-06-30 18:05:41,share,2015-06-30
72920,67947720,38455179,2015-06-30 18:24:42,share,2015-06-30
78988,41237360,38280751,2015-07-01 19:41:04,share,2013-06-29
64898,51369298,38306489,2015-07-01 21:11:15,share,2015-06-24


In [194]:
follow_follower_lists = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists_3.pkl')

def compute_indirect_return_v4(follower_lists, follows_received, shares_follow_df, indirect_return_df): 
    
    shares_follow_df['date_sent'] = shares_follow_df['date_sent'].dt.normalize()
    shares_follow_df = shares_follow_df.loc[shares_follow_df.user_id.isin(creator_ids)]
    shares_follow_df = shares_follow_df.loc[shares_follow_df.fan_id.isin(follower_lists.index.unique())]

    follows_received = follows_received.loc[follows_received.user_id.isin(creator_ids)]
    follows_received = follows_received.loc[follows_received.fan_id.isin(follower_lists.index.unique())]
    follows_received['follower_since'] = follows_received['follower_since'].dt.normalize()
    
    for user_id in tqdm(shares_follow_df.user_id.unique()):
        df_j = shares_follow_df.loc[shares_follow_df.user_id == user_id]
        
        for fan_id in shares_follow_df.loc[shares_follow_df.user_id == user_id].fan_id.unique():
            
            fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]
            
            start_date = df_j.loc[(df_j.user_id == user_id)&
                                              (df_j.fan_id == fan_id)].date_sent.iloc[0]
            end_date = datetime(2015, 7, 30)
            
            if follows_received.loc[(follows_received.user_id == user_id) &\
                (follows_received.follower_since >= start_date)].shape[0] == 0:
                pass
            else:
                creator_followers_from_t = follows_received.loc[(follows_received.user_id == user_id) &\
                                     (follows_received.follower_since >= start_date)]\
                                       .groupby('user_id', as_index = False)['fan_id'].apply(np.array)[0]
                
                indirect_return_df.loc[(indirect_return_df['user_id'] == user_id)\
                                   & (indirect_return_df['fan_id'] == fan_id), 'Follower_prop'] =\
            sum(np.in1d(creator_followers_from_t, fan_followers))/len(fan_followers)
                
                #indirect_return_df.loc[(indirect_return_df['user_id'] == user_id)
                 #                  & (indirect_return_df['fan_id'] == fan_id), 'Focal_follower_fb'] =\
                #len(fan_followers)

In [195]:
indirect_return_df2 = shares_df.loc[(shares_df.user_id.isin(creator_ids))&
                                          (shares_df.fan_id.isin(follow_follower_lists.index.unique()))
                                          ,['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])

compute_indirect_return_v4(follow_follower_lists, follows_received, shares_df, indirect_return_df2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1128/1128 [00:41<00:00, 27.51it/s]


In [197]:
#indirect_return_df2.to_csv('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/after_repost_follower_prop_all_creators.csv')

indirect_return_df2['user_type'] = indirect_return_df2.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
indirect_return_df2['creator_type'] = indirect_return_df2.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

In [201]:
indirect_return_df2.to_csv('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/after_repost_follower_prop_all_creators.csv')


In [199]:
indirect_return_df2.groupby(['user_type', 'creator_type'], as_index = False).agg({'Follower_prop': ['mean', 'median', 'std']})

user_type  creator_type Follower_prop                    
                                    mean    median       std
0     Hermit         other      0.214652  0.154902  0.262302
1     Hermit    successful      0.117551  0.000000  0.271370
2     Hermit  unsuccessful      0.062963  0.000000  0.165365
3   Observer    successful      0.010638  0.010638  0.015045
4        f_a         other      0.060179  0.000000  0.080198
5        f_a    successful      0.052744  0.000000  0.128378
6        f_a  unsuccessful      0.000000  0.000000  0.000000
7      other         other      0.074499  0.000000  0.140679
8      other    successful      0.047277  0.000000  0.114485
9      other  unsuccessful      0.047496  0.000000  0.125943
10       w_a         other      0.010034  0.000000  0.024067
11       w_a    successful      0.023991  0.007634  0.044824
12       w_a  unsuccessful      0.004667  0.000000  0.015707

In [154]:
indirect_return_df2.loc[indirect_return_df2.user_type == 'Observer']

,user_id,fan_id,Follower_prop,Focal_follower_fb,user_type,creator_type
6589,38289161,73936605,0.000000,2.0,Observer,successful
26946,37842894,45213965,0.021277,47.0,Observer,successful


In [147]:
indirect_return_df2.loc[indirect_return_df2.user_type == 'Observer'].Follower_prop.describe([.25, .5, .75, .8, .9])

count    2.000000
mean     0.010638
std      0.015045
min      0.000000
25%      0.005319
50%      0.010638
75%      0.015957
80%      0.017021
90%      0.019149
max      0.021277
Name: Follower_prop, dtype: float64

In [146]:
indirect_return_df2.loc[indirect_return_df2.user_type == 'w_a'].Follower_prop.describe([.25, .5, .75, .8, .9])

count    2801.000000
mean        0.023991
std         0.044824
min         0.000000
25%         0.000000
50%         0.007634
75%         0.027397
80%         0.034483
90%         0.067797
max         0.597039
Name: Follower_prop, dtype: float64

In [145]:
indirect_return_df2.loc[indirect_return_df2.user_type == 'f_a'].Follower_prop.describe([.25, .5, .75, .8, .9])

count    228.000000
mean       0.052744
std        0.128378
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
80%        0.059529
90%        0.217495
max        1.000000
Name: Follower_prop, dtype: float64

In [156]:
indirect_return_df2.loc[indirect_return_df2.user_type == 'f_a']

,user_id,fan_id,Follower_prop,Focal_follower_fb,user_type,creator_type
15184,38381896,33988992,0.333333,6.0,f_a,successful
4091,38091819,40120578,0.357143,42.0,f_a,successful
4104,38239638,17589406,0.272727,11.0,f_a,successful
21224,37959294,43210312,0.000000,14.0,f_a,successful
30654,37842894,40947239,0.011628,86.0,f_a,successful
...,...,...,...,...,...,...
65951,38393304,157384195,0.000000,2.0,f_a,successful
79746,38239638,53388244,0.000000,8.0,f_a,successful
70729,38230279,50238253,0.000000,95.0,f_a,successful
82871,38455179,29512962,0.000000,61.0,f_a,successful


In [157]:
indirect_return_df2.groupby('user_type', as_index = True).Focal_follower_fb.describe()

,count,mean,std,min,25%,50%,75%,max
user_type,,,,,,,,
Hermit,22.0,14.090909,22.189464,1.0,2.00,8.5,16.00,106.0
Observer,2.0,24.500000,31.819805,2.0,13.25,24.5,35.75,47.0
f_a,228.0,31.254386,128.831264,1.0,3.00,7.0,15.00,1542.0
other,3695.0,58.221110,364.880877,1.0,9.00,23.0,50.00,16488.0
w_a,2801.0,459.556230,2307.260517,4.0,89.00,166.0,349.00,66751.0


In [97]:
shares_follow_df = shares_df
follower_lists = follow_follower_lists

shares_follow_df['date_sent'] = shares_follow_df['date_sent'].dt.normalize()
shares_follow_df = shares_follow_df.loc[(shares_follow_df.user_id.isin(creator_ids))]
shares_follow_df = shares_follow_df.loc[shares_follow_df.fan_id.isin(follower_lists.index.unique())]

follows_received = follows_received.loc[follows_received.user_id.isin(successful_ids)]
follows_received = follows_received.loc[follows_received.fan_id.isin(follower_lists.index.unique())]
follows_received['follower_since'] = follows_received['follower_since'].dt.normalize()

In [98]:
shares_follow_df

,fan_id,user_id,date_sent,outbound_activity,follower_since
9620,4074557,37851487,2013-03-01,share,2013-03-01 09:07:06
17020,2564143,37872188,2013-03-01,share,2013-03-01 15:06:52
21562,18771878,37834466,2013-03-01,share,2013-03-01 23:05:43
32475,38021458,37834466,2013-03-02,share,2013-03-02 22:16:50
18501,38024424,37969361,2013-03-03,share,2013-03-03 01:13:26
...,...,...,...,...,...
69258,69069978,38455179,2015-06-30,share,2015-06-30 08:42:51
61900,2127949,38407067,2015-06-30,share,2015-01-27 22:18:24
50993,9215506,38445740,2015-06-30,share,2015-06-30 18:05:40
72920,67947720,38455179,2015-06-30,share,2015-06-30 18:18:09


In [111]:
user_id = 37969361
fan_id = 38024424

df_j = shares_follow_df.loc[shares_follow_df.user_id == user_id]

fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]
start_date = df_j.loc[(df_j.user_id == user_id)&
                                  (df_j.fan_id == fan_id)].date_sent.iloc[0]
end_date = datetime(2015, 7, 30)

In [112]:
creator_followers_from_t = follows_received.loc[follows_received.follower_since >= start_date]\
                                       .groupby('user_id', as_index = False)['fan_id'].apply(np.array)[0]

In [113]:
creator_followers_from_t

array([ 23642821,  30596511,  38181109,  25664035,   8777948,  34210646,
         7683426,  42585503,  10573441,  19646071,  37266830,    119764,
        42433503,  28985630,   2459357,  21725063,  15206558,  33306039,
        21639437,  38962022,   6262728,  39438417,  37891621,   6510542,
        36897576,  18745143,  26191450,  39613696,  39262514,  40759678,
        42799219,  35263385,  26844131,  17970160,  15636675,  30614653,
         8074149,  32775881,  23700609,   7986128,   4632804,  41273005,
        31285741,  38533102,  37390003,   9419504,  38160038,   4483316,
        18800348,  35821334,  38364609,   1990688,  34209775,  44194064,
        30563110,  11287107,  46294297,  39984031,  34713868,  45338386,
        34219311,  43301807,  19076610,   7460935,   1297446,   9738790,
        42571394,  26190014,  22308483,   3791289,   9714644,  40005393,
        33742165,  20643089,  39463042,   9172790,   7678828,  24907888,
        42335354,  37261943,   5096913,   8740995, 

In [114]:
fan_followers

array([ 38154564,  39435460,  41569219,  42028125,  43333433,  46001133,
        45798760,  48819238,  49515916,  50117066,  50319765,  55080556,
        41692773,  60126073,  73037332,  77058102,  80239488,  52183508,
       103534701, 101923317, 104061597,  36683295, 102816453, 127171086],
      dtype=int32)

In [115]:
sum(np.in1d(creator_followers_from_t, fan_followers))/len(fan_followers)

0.0

In [121]:
indirect_return_df2

,user_id,fan_id,Follower_prop
9620,37851487,4074557,0.002088
17020,37872188,2564143,0.000000
21562,37834466,18771878,0.000000
32475,37834466,38021458,0.000000
18501,37969361,38024424,0.000000
...,...,...,...
69258,38455179,69069978,0.000000
61900,38407067,2127949,0.000000
50993,38445740,9215506,0.000000
72920,38455179,67947720,0.000000


In [122]:
max(indirect_return_df2.Follower_prop)

0.06666666666666667

In [138]:
indirect_return_df2

,user_id,fan_id,Follower_prop,user_type,creator_type
9620,37851487,4074557,0.022965,w_a,successful
17020,37872188,2564143,0.009174,w_a,successful
21562,37834466,18771878,0.018182,other,successful
32475,37834466,38021458,0.000000,other,successful
18501,37969361,38024424,0.041667,other,successful
...,...,...,...,...,...
69258,38455179,69069978,0.000000,w_a,successful
61900,38407067,2127949,0.001805,other,successful
50993,38445740,9215506,0.000000,other,successful
72920,38455179,67947720,0.000000,other,successful


In [202]:
indirect_return_df

,user_id,fan_id,Follower_prop,user_type,creator_type
0,37873923,2763368,0.007407,other,other
1,37876767,32678887,0.000000,other,other
4,37873923,3007254,0.000000,other,other
6,37929708,30570703,0.144444,other,other
7,37918716,7935937,0.019802,w_a,successful
...,...,...,...,...,...
432482,38393304,112388565,0.000000,w_a,successful
432483,38239638,20904612,0.000000,w_a,successful
432488,38141992,160442680,0.000000,other,other
432500,38222799,90937808,0.000683,w_a,successful
